In [ ]:
## Ce code permet d'améliorer la mise en page du notebook

from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML("<style>"+styles+"</style>")
css_styling()

# Décorateurs et décoration

## Introduction

Les décorateurs font très probablement partie des possibilités de conception les plus belles et les plus puissantes de Python, mais en même temps le concept est considéré par beaucoup comme compliqué à aborder. Pour être précis, l'utilisation des décorateurs est très facile, mais l'écriture des décorateurs peut être compliquée, surtout si vous n'avez pas d'expérience avec les décorateurs et certains concepts de programmation fonctionnelle.

Bien qu'il s'agisse du même concept sous-jacent, il existe deux types différents de décorateurs en Python :

- Les décorateurs de fonctions
- Les décorateurs de classe

Un décorateur en Python est tout objet Python appelable qui est utilisé pour modifier une fonction ou une classe. Une référence à une fonction ```func``` ou une classe ```C``` est passée à un décorateur et le décorateur retourne une fonction ou une classe modifiée. Les fonctions ou classes modifiées contiennent généralement des appels à la fonction originale ```func``` ou à la classe ```C```.

## Premiers pas vers les décorateurs

Nous allons présenter les décorateurs en reprenant certains aspects importants des fonctions. Tout d'abord, vous devez savoir ou vous souvenir que les noms de fonctions sont des références aux fonctions et que nous pouvons attribuer plusieurs noms à la même fonction :

In [1]:
def succ(x):
    return x + 1
successor = succ
successor(10)

11

Cela signifie que nous avons deux noms, à savoir ```succ``` et ```successeur``` pour la même fonction. Le fait important suivant est que nous pouvons supprimer soit ```succ``` soit ```successeur``` sans supprimer la fonction elle-même.

In [2]:
del succ
successor(10)

11

### Fonctions dans les fonctions
Le concept d'avoir ou de définir des fonctions à l'intérieur d'une fonction est complètement nouveau pour les programmeurs C ou C++ :

In [3]:
def f():
    
    def g():
        print("Hi, it's me 'g'")
        print("Thanks for calling me")
        
    print("This is the function 'f'")
    print("I am calling 'g' now:")
    g()

    
f()

This is the function 'f'
I am calling 'g' now:
Hi, it's me 'g'
Thanks for calling me


Un autre exemple d'utilisation de déclarations de retour "appropriées" dans les fonctions :

In [4]:
def temperature(t):
    def celsius2fahrenheit(x):
        return 9 * x / 5 + 32

    result = "It's " + str(celsius2fahrenheit(t)) + " degrees!" 
    return result

print(temperature(20))

It's 68.0 degrees!


L'exemple suivant concerne la fonction factorielle, que nous avons précédemment définie comme suit :

In [5]:
def factorial(n):
    """ calculates the factorial of n, 
        n should be an integer and n <= 0 """
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

Que se passe-t-il si quelqu'un passe une valeur négative ou un nombre flottant à cette fonction ? Elle ne se terminera jamais. Vous pourriez avoir l'idée de vérifier cela comme suit :

In [6]:
def factorial(n):
    """ calculates the factorial of n, 
        n should be an integer and n <= 0 """
    if type(n) == int and n >=0:
        if n == 0:
            return 1
        else:
            return n * factorial(n-1)

    else:
        raise TypeError("n has to be a positive integer or zero")

Si vous appelez cette fonction avec 4 par exemple, c'est-à-dire factorielle (4), la première chose qui est vérifiée est de savoir si c'est mon entier positif. En principe, cela a du sens. Le "problème" apparaît maintenant dans l'étape de récursion. On appelle maintenant la factorielle (3). Cet appel et tous les autres vérifient également s'il s'agit d'un nombre entier positif. Mais cela n'est pas nécessaire : Si vous soustrayez la valeur 1 d'un nombre entier positif, vous obtenez à nouveau un nombre entier positif ou 0. Les deux valeurs d'argument sont donc bien définies pour notre fonction.

Avec une fonction imbriquée (fonction locale), on peut résoudre ce problème de manière élégante :

In [7]:
def factorial(n):
    """ calculates the factorial of n, 
        n should be an integer and n <= 0 """
    def inner_factorial(n):
        if n == 0:
            return 1
        else:
            return n * inner_factorial(n-1)
        
    if type(n) == int and n >=0:
        return inner_factorial(n)
    else:
        raise TypeError("n should be a positve int or 0")

Nous pouvons étendre le domaine des valeurs d'entrée possibles pour notre fonction factorielle en autorisant les nombres flottants qui sont équivalents aux nombres entiers, c'est-à-dire qui satisfont la condition ```int(x) == x```. Si nous savons qu'une variable ```x``` fait référence à une valeur flottante, nous pouvons également utiliser le test ```x.is_integer()```.

L'implémentation suivante de factorial suit une analyse de cas plus détaillée de l'argument comme discuté précédemment :

In [8]:
def factorial(n):
    """ calculates the factorial of n, if n is either a non negative
    integer or a float number x being equivalent to an integer, like
    4.0, 12.0, 8. i.e. no decimals following the decimal point """
    
    def inner_factorial(n):
        if n == 0:
            return 1
        else:
            return n * inner_factorial(n-1)
        
    if not isinstance(n, (int, float)):
        raise ValueError("Value is neither an integer nor a float equivalent to int")
    if isinstance(n, (int))  and n < 0:
        raise ValueError('Should be a positive integer or 0')
    elif isinstance(n, (float)) and not n.is_integer():
        raise ValueError('value is a float but not equivalent to an int')
    else:
        return inner_factorial(n)

Vérifions la fonction précédente :

In [9]:
values = [0, 1, 5, 7.0, -4, 7.3, "7"]

for value in values:
    try: 
        print(value, end=", ")
        print(factorial(value))
    except ValueError as e:
        print(e)

0, 1
1, 1
5, 120
7.0, 5040.0
-4, Should be a positive integer or 0
7.3, value is a float but not equivalent to an int
7, Value is neither an integer nor a float equivalent to int


### Fonctions en tant que paramètres
Si vous regardez uniquement les exemples précédents, cela ne semble pas très utile. Cela devient utile en combinaison avec deux autres possibilités puissantes des fonctions Python. Étant donné que chaque paramètre d'une fonction est une référence à un objet et que les fonctions sont également des objets, nous pouvons passer des fonctions - ou mieux des "références à des fonctions" - comme paramètres à une fonction. Nous allons le démontrer dans le prochain exemple simple :

In [10]:
def g():
    print("Hi, it's me 'g'")
    print("Thanks for calling me")
    
def f(func):
    print("Hi, it's me 'f'")
    print("I will call 'func' now")
    func()
          
f(g)

Hi, it's me 'f'
I will call 'func' now
Hi, it's me 'g'
Thanks for calling me


Vous pouvez ne pas être satisfait du résultat. ```f``` devrait écrire qu'il appelle ```g``` et non ```func```. Bien sûr, nous devons savoir quel est le "vrai" nom de ```func```. Pour cela, nous pouvons utiliser l'attribut ```__name__```, car il contient ce nom :

In [11]:
def g():
    print("Hi, it's me 'g'")
    print("Thanks for calling me")
    
def f(func):
    print("Hi, it's me 'f'")
    print("I will call 'func' now")
    func()
    print("func's real name is " + func.__name__) 

          
f(g)

Hi, it's me 'f'
I will call 'func' now
Hi, it's me 'g'
Thanks for calling me
func's real name is g


La sortie explique à nouveau ce qui se passe. Un autre exemple :

In [12]:
import math

def foo(func):
    print("The function " + func.__name__ + " was passed to foo")
    res = 0
    for x in [1, 2, 2.5]:
        res += func(x)
    return res

print(foo(math.sin))
print(foo(math.cos))

The function sin was passed to foo
2.3492405557375347
The function cos was passed to foo
-0.6769881462259364


### Fonctions renvoyant des fonctions
La sortie d'une fonction est également une référence à un objet. Les fonctions peuvent donc renvoyer des références à des objets fonctionnels.

In [13]:
def f(x):
    def g(y):
        return y + x + 3 
    return g

nf1 = f(1)
nf2 = f(3)

print(nf1(1))
print(nf2(1))

5
7


L'exemple précédent semble très artificiel et absolument inutile. Nous allons maintenant présenter un autre exemple orienté langage, qui montre une touche plus pratique. Bon, toujours pas une fonction qui soit utile telle quelle. Nous écrivons une fonction avec le nom presque explicite ```greeting_func_gen```. Cette fonction renvoie (ou génère) des fonctions qui peuvent être utilisées pour créer des personnes dans différentes langues, à savoir l'allemand, le français, l'italien, le turc et le grec :

In [14]:
def greeting_func_gen(lang):
    
    def customized_greeting(name):
        if lang == "de":   # German
            phrase = "Guten Morgen "
        elif lang == "fr": # French
            phrase = "Bonjour "
        elif lang == "it": # Italian
            phrase = "Buongiorno "
        elif lang == "tr": # Turkish
            phrase = "Günaydın "
        elif lang == "gr": # Greek
            phrase = "Καλημερα "
        else:
            phrase = "Hi "
        return phrase + name + "!"
    
    return customized_greeting


say_hi = greeting_func_gen("tr")
print(say_hi("Gülay"))    # this Turkish name means "rose moon" by the way

Günaydın Gülay!


## Un exemple plus utile

L'exemple suivant est plus utile et en même temps plus orienté vers les mathématiques. Supposons que nous devions définir plusieurs polynômes de degré 2. Cela peut ressembler à ceci :

In [15]:
def p1(x):
    return 2*x**2 - 3*x + 0.5

def p2(x):
    return 2.3*x**2 + 2.9*x - 20

def p3(x):
    return -2.3*x**2 + 4.9*x - 9

Cela peut être simplifié en implémentant dès maintenant une fonction de "fabrique" de polynômes. Nous allons commencer par écrire une version qui peut créer des polynômes de degré 2.
<center>$p(x) = ax^2+bx+c$</center>


L'implémentation Python en tant que fonction de fabrique polynomiale peut être écrite comme suit :

In [16]:
def polynomial_creator(a, b, c):
    def polynomial(x):
        return a * x**2 + b * x + c
    return polynomial
    
p1 = polynomial_creator(2, -3, 0.5)
p2 = polynomial_creator(2.3, 2.9, -20)
p3 = polynomial_creator(-2.3, 4.9, -9)

for x in range(-2, 2, 1):
    print(x, p1(x), p2(x))

-2 14.5 -16.6
-1 5.5 -20.6
0 0.5 -20.0
1 -0.5 -14.8


Nous pouvons généraliser notre fonction de fabrique afin qu'elle puisse fonctionner pour des polynômes de degré arbitraire :

<center>$\sum_{k=0}^n a_kx^k=a_kx^k+a_{k-1}x^{k+1}+\cdots + a_1x^1+a_0$</center>

In [17]:
def polynomial_creator(*coefficients):
    """ coefficients are in the form a_n, ... a_1, a_0 
    """
    def polynomial(x):
        res = 0
        for index, coeff in enumerate(coefficients[::-1]):
            res += coeff * x** index
        return res
    return polynomial
  
p1 = polynomial_creator(4)
p2 = polynomial_creator(2, 4)
p3 = polynomial_creator(1, 8, -1, 0, 3, 2)
p4 = polynomial_creator(-1, 2, 1)
p5 = polynomial_creator(4, 5, 7, 7, 9, 12, 3, 43, 9)


for x in range(-2, 2, 1):
    print(x, p1(x), p2(x), p3(x), p4(x), p5(x))

-2 4 0 100 -7 591
-1 4 2 7 -2 -35
0 4 4 2 1 9
1 4 6 13 2 99


La fonction $p_3$ met en œuvre, par exemple, le polynôme suivant :
$p_3(x)=x^5+8x^4-x^3+3x+2$

La fonction polynomiale à l'intérieur de notre décorateur polynomial_creator peut être implémentée plus efficacement. Nous pouvons la factoriser de manière à ce qu'elle ne nécessite aucune exponentiation.

Version factorisée d'un polynôme général sans exponentiation :

<center>$res=((a_nx +a_{n-1}x + \cdots +a_1)x +a_0 $</center>

Implémentation de notre décorateur de créateur polynomial évitant l'exponentiation :

In [18]:
def polynomial_creator(*coeffs):
    """ coefficients are in the form a_n, a_n_1, ... a_1, a_0 
    """
    def polynomial(x):
        res = coeffs[0]
        for i in range(1, len(coeffs)):
            res = res * x + coeffs[i]
        return res
                 
    return polynomial

p1 = polynomial_creator(4)
p2 = polynomial_creator(2, 4)
p3 = polynomial_creator(1, 8, -1, 0, 3, 2)
p4 = polynomial_creator(-1, 2, 1)
p5 = polynomial_creator(4, 5, 7, 7, 9, 12, 3, 43, 9)


for x in range(-2, 2, 1):
    print(x, p1(x), p2(x), p3(x), p4(x), p5(x))

-2 4 0 100 -7 591
-1 4 2 7 -2 -35
0 4 4 2 1 9
1 4 6 13 2 99


## Un décorateur simple

Nous avons maintenant tout préparé pour définir notre premier décorateur simple :

In [19]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        func(x)
        print("After calling " + func.__name__)
    return function_wrapper

def foo(x):
    print("Hi, foo has been called with " + str(x))

print("We call foo before decoration:")
foo("Hi")
    
print("We now decorate foo with f:")
foo = our_decorator(foo)

print("We call foo after decoration:")
foo(42)

We call foo before decoration:
Hi, foo has been called with Hi
We now decorate foo with f:
We call foo after decoration:
Before calling foo
Hi, foo has been called with 42
After calling foo


Si vous regardez la sortie du programme précédent, vous pouvez voir ce qui se passe. Après la décoration "foo = our_decorator(foo)", foo est une référence à la 'function_wrapper'. foo' sera appelé à l'intérieur de 'function_wrapper', mais avant et après l'appel, du code supplémentaire sera exécuté, c'est-à-dire, dans notre cas, deux fonctions print.

## La syntaxe habituelle des décorateurs en Python

La décoration en Python n'est généralement pas effectuée de la manière dont nous l'avons fait dans l'exemple précédent, même si la notation ```foo = our_decorator(foo)``` est accrocheuse et facile à comprendre. C'est la raison pour laquelle nous l'avons utilisée ! Vous pouvez également constater un problème de conception dans notre approche précédente. ```foo``` existait dans le même programme en deux versions, avant la décoration et après la décoration.

Nous allons faire une décoration correcte maintenant. La décoration se produit dans la ligne précédant l'en-tête de la fonction. Le __@__ est suivi du nom de la fonction décoratrice.

Nous allons maintenant réécrire notre exemple initial. Au lieu d'écrire l'instruction
```python
 foo = notre_décorateur(foo)
```

nous pouvons écrire
```python
 @notre_décorateur 
```
Mais cette ligne doit être placée directement devant la fonction décorée. L'exemple complet ressemble maintenant à ceci :

In [20]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        func(x)
        print("After calling " + func.__name__)
    return function_wrapper

@our_decorator
def foo(x):
    print("Hi, foo has been called with " + str(x))

foo("Hi")

Before calling foo
Hi, foo has been called with Hi
After calling foo


Nous pouvons décorer toutes les autres fonctions qui prennent un paramètre avec notre décorateur ```our_decorator```. Nous en faisons la démonstration dans l'exemple suivant. Nous avons légèrement modifié notre enveloppe de fonction, afin de pouvoir voir le résultat des appels de fonction :

In [21]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        res = func(x)
        print(res)
        print("After calling " + func.__name__)
    return function_wrapper

@our_decorator
def succ(n):
    return n + 1

succ(10)

Before calling succ
11
After calling succ


Il est également possible de décorer des fonctions tierces, par exemple des fonctions que nous importons d'un module. Dans ce cas, nous ne pouvons pas utiliser la syntaxe Python avec le signe ```@``` :

In [22]:
from math import sin, cos

def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        res = func(x)
        print(res)
        print("After calling " + func.__name__)
    return function_wrapper

sin = our_decorator(sin)
cos = our_decorator(cos)

for f in [sin, cos]:
    f(3.1415)

Before calling sin
9.265358966049024e-05
After calling sin
Before calling cos
-0.9999999957076562
After calling cos


## Extension des fonctions trigonométriques de maths

Créons un décorateur plus utile pour les fonctions trigonométriques. Si vous regardez l'aide de ```sin```, ```cos``` ou les autres fonctions trigonométriques du module ```math```, vous verrez que les arguments doivent être des valeurs radiales. Et si vous souhaitez utiliser des degrés ? Dans ce cas, vous devrez transformer les valeurs en radians.

In [23]:
from math import sin, cos, pi
help(sin)

Help on built-in function sin in module math:

sin(x, /)
    Return the sine of x (measured in radians).



In [24]:
angle = 45   # degrees
x = angle * pi / 180   # degrees into radians
x

0.7853981633974483

Maintenant, nous pouvons appliquer la valeur à ```sin``` ou à d'autres fonctions trigonométriques :

In [25]:
sin(x)

0.7071067811865475

Nous pourrions également étendre les fonctions trigonométriques avec un décorateur, qui transforme automatiquement les degrés en radians. Nous ajoutons un autre paramètre aux fonctions.

In [26]:
from math import sin, cos, pi
def angle_deco(func):
    
    def helper(x, mode="radians"):
        if mode == "degrees":
            x = x * pi / 180
        return func(x)
    
    return helper

sin = angle_deco(sin)
cos = angle_deco(cos)

In [27]:
degrees = [40, 45, 70, 90]
for degree in degrees:
    print(sin(degree, mode='degrees'), cos(degree, mode='degrees'))

0.6427876096865393 0.7660444431189781
0.7071067811865475 0.7071067811865476
0.9396926207859083 0.3420201433256688
1.0 6.123233995736766e-17


En résumé, on peut dire qu'un décorateur en Python est un objet Python appelable qui est utilisé pour modifier une fonction, une méthode ou une définition de classe. L'objet original, celui qui va être modifié, est passé à un décorateur comme argument. Le décorateur retourne un objet modifié, par exemple une fonction modifiée, qui est liée au nom utilisé dans la définition.

Le précédent ```function_wrapper``` ne fonctionne que pour les fonctions ayant exactement un paramètre. Nous fournissons une version généralisée du ```function_wrapper```, qui accepte les fonctions avec des paramètres arbitraires dans l'exemple suivant :

In [28]:
from random import random, randint, choice

def our_decorator(func):
    def function_wrapper(*args, **kwargs):
        print("Before calling " + func.__name__)
        res = func(*args, **kwargs)
        print(res)
        print("After calling " + func.__name__)
    return function_wrapper

random = our_decorator(random)
randint = our_decorator(randint)
choice = our_decorator(choice)

random()
randint(3, 8)
choice([4, 5, 6])

Before calling random
0.2841650963613198
After calling random
Before calling randint
5
After calling randint
Before calling choice
6
After calling choice


## Utilisation de décorateurs multiples

Il est possible de décorer des fonctions avec plus d'un décorateur.

In [29]:
def deco1(func):
    
    print('deco1 has been called')
    def helper(x):
        print('helper of deco1 has been called!')
        print(x)
        return func(x) + 3
    return helper
    
def deco2(func):
    
    print('deco2 has been called')
    def helper(x):
        print('helper of deco2 has been called!')
        print(x)
        return func(x) + 2
    return helper
    
def deco3(func):
    
    print('deco3 has been called')
    def helper(x):
        print('helper of deco3 has been called!')
        print(x)
        return func(x) + 1
    return helper
    
@deco3
@deco2
@deco1
def foobar(x):
    return 42

deco1 has been called
deco2 has been called
deco3 has been called


Le résultat nous montre que la fonction ```foobar``` est d'abord décorée avec ```deco1```, c'est-à-dire le décorateur directement au-dessus de la définition de la fonction. Ensuite, elle est décorée avec le décorateur 2, puis le décorateur 3.

Lorsque nous appelons la fonction décorée plusieurs fois, cela fonctionne dans l'autre sens `

In [30]:
foobar(42)

helper of deco3 has been called!
42
helper of deco2 has been called!
42
helper of deco1 has been called!
42


48

## Cas d'utilisation des décorateurs

### Vérifier les arguments avec un décorateur
Reprenons le cas de la fonction factorielle. Nous voulons garder la fonction aussi simple que possible et nous ne voulons pas masquer l'idée sous-jacente, donc nous n'avons pas incorporé de vérification des arguments. Ainsi, si quelqu'un appelle notre fonction avec un argument négatif ou avec un argument flottant, notre fonction entre dans une boucle sans fin.

Le programme suivant utilise une fonction décorateur pour s'assurer que l'argument passé à la fonction factorielle est un entier positif :

In [31]:
def argument_test_natural_number(f):
    def helper(x):
        if type(x) == int and x > 0:
            return f(x)
        else:
            raise ValueError("Argument is not an integer")
    return helper

@argument_test_natural_number
def is_prime(n):
    return all(n % i for i in range(2, n))

for i in range(1,10):
    print(i, is_prime(i))

try:
    print(is_prime(-1))
except ValueError:
    print("Argument is not a positve integer!")

1 True
2 True
3 True
4 False
5 True
6 False
7 True
8 False
9 False
Argument is not a positve integer!


### Compter les appels de fonction avec les décorateurs
L'exemple suivant utilise un décorateur pour compter le nombre de fois qu'une fonction a été appelée. Pour être précis, nous pouvons utiliser ce décorateur uniquement pour les fonctions ayant exactement un paramètre :

In [32]:
def call_counter(func):
    def helper(x):
        helper.calls += 1
        return func(x)
    helper.calls = 0

    return helper

@call_counter
def succ(x):
    return x + 1

print(succ.calls)
for i in range(10):
    succ(i)
    
print(succ.calls)

0
10


Nous avons souligné que nous ne pouvons utiliser notre décorateur précédent que pour les fonctions qui prennent exactement un paramètre. Nous utiliserons les notations ```*args``` et ```**kwargs``` pour écrire des décorateurs capables de gérer des fonctions avec un nombre arbitraire de paramètres positionnels et de mots-clés.

In [33]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0

    return helper

@call_counter
def succ(x):
    return x + 1

@call_counter
def mul1(x, y=1):
    return x*y + 1

print(succ.calls)
for i in range(10):
    succ(i)
mul1(3, 4)
mul1(4)
mul1(y=3, x=2)
    
print(succ.calls)
print(mul1.calls)

0
10
3


## Décorateurs avec paramètres

Nous définissons deux décorateurs dans le code suivant :

In [34]:
def evening_greeting(func):
    def function_wrapper(x):
        print("Good evening, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper

def morning_greeting(func):
    def function_wrapper(x):
        print("Good morning, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper

@evening_greeting
def foo(x):
    print(42)

foo("Hi")

Good evening, foo returns:
42


Ces deux décorateurs sont presque identiques, sauf pour le message d'accueil. Nous voulons ajouter un paramètre au décorateur pour être capable de personnaliser la salutation, lorsque nous faisons la décoration. Nous devons envelopper une autre fonction autour de notre fonction de décorateur précédente pour accomplir ceci. Nous pouvons maintenant facilement dire "Good Morning" en grec :

In [35]:
def greeting(expr):
    def greeting_decorator(func):
        def function_wrapper(x):
            print(expr + ", " + func.__name__ + " returns:")
            func(x)
        return function_wrapper
    return greeting_decorator

@greeting("καλημερα")
def foo(x):
    print(42)

foo("Hi")

καλημερα, foo returns:
42


Si nous ne voulons pas ou ne pouvons pas utiliser la syntaxe du décorateur ```@```, nous pouvons le faire avec des appels de fonction :

In [36]:
def greeting(expr):
    def greeting_decorator(func):
        def function_wrapper(x):
            print(expr + ", " + func.__name__ + " returns:")
            return func(x)
        return function_wrapper
    return greeting_decorator


def foo(x):
    print(42)

greeting2 = greeting("καλημερα")
foo = greeting2(foo)
foo("Hi")

καλημερα, foo returns:
42


Bien sûr, nous n'avons pas besoin de la définition supplémentaire de ```greeting2```. Nous pouvons directement appliquer le résultat de l'appel ```greeting("καλημερα")``` sur ```foo``` :
```python
foo = greeting("καλημερα")(foo)
```

## Utilisation des wrappers de functools
La façon dont nous avons défini les décorateurs jusqu'à présent n'a pas pris en compte le fait que les attributs

- ```__name__``` (le nom de la fonction);
- ```__doc__``` (la docstring);
- ```__module__``` (Le module dans lequel la fonction est définie)

des fonctions originales seront perdues après la décoration.

In [37]:
def greeting(func):
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper 

Nous l'appelons dans le programme suivant :

In [38]:
@greeting
def f(x):
    """ just some silly function """
    return x + 4

f(10)
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__) 

Hi, f returns:
function name: function_wrapper
docstring:  function_wrapper of greeting 
module name: __main__


Nous obtenons les résultats __indésirables__ ci-dessus.

Nous pouvons sauvegarder les attributs originaux de la fonction ```f```, si nous les assignons à l'intérieur du décorateur. Nous modifions notre décorateur précédent en conséquence :

In [40]:
def greeting(func):
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    function_wrapper.__name__ = func.__name__
    function_wrapper.__doc__ = func.__doc__
    function_wrapper.__module__ = func.__module__
    return function_wrapper

In [41]:
@greeting
def f(x):
    """ just some silly function """
    return x + 4

f(10)
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__) 

Hi, f returns:
function name: f
docstring:  just some silly function 
module name: __main__


Heureusement, nous n'avons pas besoin d'ajouter tout ce code à nos décorateurs pour obtenir ces résultats. Nous pouvons importer le décorateur ```wraps``` de ```functools``` à la place et décorer notre fonction dans le décorateur avec celui-ci :

In [42]:
from functools import wraps

def greeting(func):
    @wraps(func)
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper
@greeting
def f(x):
    """ just some silly function """
    return x + 4

f(10)
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__) 

Hi, f returns:
function name: f
docstring:  just some silly function 
module name: __main__


## Des classes au lieu de fonctions

### La méthode d'appel
Jusqu'à présent, nous avons utilisé des fonctions comme décorateurs. Avant de pouvoir définir un décorateur comme une classe, nous devons introduire la méthode ```__call__``` des classes. Nous avons déjà mentionné qu'un décorateur est simplement un objet appelable qui prend une fonction comme paramètre d'entrée. Une fonction est un objet appelable, mais beaucoup de programmeurs Python ne savent pas qu'il existe d'autres objets appelables. Un objet appelable est un objet qui peut être utilisé et se comporter comme une fonction, mais qui peut ne pas être une fonction. Il est possible de définir des classes de manière à ce que les instances soient des objets appelables. La méthode ```__call__``` est appelée, si l'instance est appelée __comme une fonction__, c'est-à-dire en utilisant des parenthèses.

In [43]:
class A:
    def __init__(self):
        print("An instance of A was initialized")
    
    def __call__(self, *args, **kwargs):
        print("Arguments are:", args, kwargs)
              
x = A()
print("now calling the instance:")
x(3, 4, x=11, y=10)
print("Let's call it again:")
x(3, 4, x=11, y=10)

An instance of A was initialized
now calling the instance:
Arguments are: (3, 4) {'x': 11, 'y': 10}
Let's call it again:
Arguments are: (3, 4) {'x': 11, 'y': 10}


Nous pouvons écrire une classe pour la fonction fibonacci en utilisant la méthode ```__call__``` :

In [44]:
class Fibonacci:

    def __init__(self):
        self.cache = {}

    def __call__(self, n):
        if n not in self.cache:
            if n == 0:
                self.cache[0] = 0
            elif n == 1:
                self.cache[1] = 1
            else:
                self.cache[n] = self.__call__(n-1) + self.__call__(n-2)
        return self.cache[n]

fib = Fibonacci()

for i in range(15):
    print(fib(i), end=", ")

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 

### Utilisation d'une classe comme décorateur
Nous allons réécrire le décorateur suivant comme une classe :

In [45]:
def decorator1(f):
    def helper():
        print("Decorating", f.__name__)
        f()
    return helper

@decorator1
def foo():
    print("inside foo()")

foo()

Decorating foo
inside foo()


Le décorateur suivant, implémenté en tant que classe, fait le même travail :

In [46]:
class decorator2:
    
    def __init__(self, f):
        self.f = f
        
    def __call__(self):
        print("Decorating", self.f.__name__)
        self.f()

@decorator2
def foo():
    print("inside foo()")

foo() 

Decorating foo
inside foo()


Les deux versions renvoient le même résultat.